# Model Prediction Verification

This script demonstrates how to train a single model class, embed the model, and solve the optimization problem. We fix a sample from our generated data and solve the optimization problem with all elements of $\mathbf{x}$ equal to our data. In general, we might have some elements of $\mathbf{x}$ that are fixed, called our "conceptual variables," and the remaining indices are our decision variables. By fixing all elements of $\mathbf{x}$, we can verify that the model prediction matches the original sklearn model.

## Load the relevant packages

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.utils.extmath import cartesian
import time
import sys
import os
import time

from sklearn.metrics import roc_auc_score, r2_score, mean_squared_error
from sklearn.cluster import KMeans

In [2]:
from importlib import reload

In [3]:
sys.path.append(os.path.abspath('../../opticl'))  # TODO: has to be changed
import constraint_learning as cl
import embed_mip as em
import run_MLmodels as ml

In [4]:
# import opticl
from pyomo import environ
from pyomo.environ import *

## Initialize data
We will work with a basic dataset from `sklearn`.

In [5]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
X, y = make_classification(n_samples=200, n_features = 20, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=1)
X_train = pd.DataFrame(X_train).add_prefix('col')
X_test = pd.DataFrame(X_test).add_prefix('col')

## Train the chosen model type

In [9]:
alg = 'rf' 
# alg_run = 'rf_shallow'
# alg = alg_run = 'mlp'
task_type = 'binary'

The user can optionally select a manual parameter grid for the cross-validation procedure. We implement a default parameter grid; see **run_MLmodels.py** for details on the tuned parameters. If you wish to use the default, leave ```parameter_grid = None``` (or do not specify any grid).

In [10]:
parameter_grid = None
# parameter_grid = {'hidden_layer_sizes': [(5,),(10,)]}

In [11]:
s = 1
version = 'test'
outcome = 'temp'

model_save = 'results/%s/%s_%s_model.csv' % (alg, version, outcome)


alg_run = alg if alg != 'rf' else 'rf_shallow'
m, perf = ml.run_model(X_train, y_train, X_test, y_test, alg_run, outcome, task = task_type, 
                       seed = s, cv_folds = 5, 
                       # The user can manually specify the parameter grid for cross-validation if desired
                       parameter_grid = parameter_grid,
                       save_path = model_save,
                       save = False)

------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = rf_shallow, metric = None
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train Score: 0.9982142857142857
-------------------testing evaluation-----------------------
Test Score: 0.8653530377668308


After training the model, we will save the trained model in the format needed for embedding the constraints. See **constraint_learning.py** for the specific format that is extracted per method. We also save the performance of the model to use in the automated model selection pipeline (if desired).

We also create the save directory if it does not exist.



In [12]:
if not os.path.exists('results/%s/' % alg):
    os.makedirs('results/%s/' % alg)
    
constraintL = cl.ConstraintLearning(X_train, y_train, m, alg)
constraint_add = constraintL.constraint_extrapolation(task_type)
constraint_add.to_csv(model_save, index = False)
# 
perf.to_csv('results/%s/%s_%s_performance.csv' % (alg, version, outcome), index= False)

### Check: what should the result be for our sample observation, if all x are fixed?

#### Choose sample to test
This will be the observation ("patient") that we feed into the optimization model.

In [13]:
sample_id = 0
sample = X_train.loc[sample_id:sample_id,:].reset_index(drop = True)

Calculate model prediction directly in sklearn.

In [14]:
m.predict_proba(sample)

array([[0.6153694, 0.3846306]])

## Optimization formulation
We will embed the model trained above. The model could also be selected using the model selection pipeline, which we demonstrate in the WFP example script.

If manually specifying the model, as we are here, the key elements of the ``model_master`` dataframe are:
- model_type: algorithm name.
- outcome: name of outcome of interest; this is relevant in the case of multiple learned outcomes.
- save_path: file name of the extracted model.
- objective: the weight of the objective if it should be included as an additive term in the objective. A weight of 0 omits it from the objective entirely.
- lb/ub: the lower (or upper) bound that we wish to apply to the learned outcome. If there is no bound, it should be set to ``None``.

In this case, we set the outcome to be our only objective term, which will allow us to verify that the predictions are consistent between the embedded model and the sklearn prediction function.

In [15]:
model_master = pd.DataFrame(columns = ['model_type','outcome','save_path','lb','ub','objective'])

model_master.loc[0,'model_type'] = alg
model_master.loc[0,'save_path'] = 'results/%s/%s_%s_model.csv' % (alg, version, outcome)
model_master.loc[0,'outcome'] = outcome
model_master.loc[0,'objective'] = 1
model_master.loc[0,'ub'] = None
model_master.loc[0,'lb'] = None
model_master.loc[0,'task'] = task_type

#### Pyomo

In [16]:
model_pyo = ConcreteModel()

## We will create our x decision variables, and fix them all to our sample's values for model verification.
N = X_train.columns
model_pyo.x = Var(N, domain=Reals)

def fix_value(model_pyo, index):
    return model_pyo.x[index] == sample.loc[0,index]

model_pyo.Constraint1 = Constraint(N, rule=fix_value)

## Specify any non-learned objective components - none here 
model_pyo.OBJ = Objective(expr=0, sense=minimize)

In [17]:
final_model_pyo = em.optimization_MIP(model_pyo, model_pyo.x, model_master, X_train, tr = False)

Embedding objective function for temp


In [18]:
opt = SolverFactory('gurobi')
results = opt.solve(final_model_pyo) 

### Check for equality between sklearn and embedded models

In [19]:
print("True outcome: %.3f" % np.array(m.predict_proba(sample))[0,1])
print("Pyomo output: %.3f" % final_model_pyo.OBJ())

True outcome: 0.385
Pyomo output: 0.385
